# Mathematical Operations

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.DataFrame(np.ones([5,4]), columns=['a', 'b', 'c', 'd'])
df

In [ ]:
# Basic math operations on a DataFrame perform the computation for every cell
df *= 2
df

In [ ]:
# You can also do calculations on specific rows or columns
df.loc[1] /= 2
df['b'] -= 1
df

In [ ]:
df2 = pd.DataFrame(np.ones([3,2]), columns=['d', 'e'], index=[2,4,5])
df2

In [ ]:
# Operating on two DataFrames:
# NaN for every combination of index/column that is not present in both inputs
df + df2

In [ ]:
# The same is true for operations on two Series
df.loc[2] * df2.loc[5]

In [ ]:
df

In [ ]:
df.mean()

In [ ]:
# This is an operation on a DataFrame and a Series
# Series indices are matched on DataFrame column labels
df - df.mean()

In [ ]:
# Another operation on DataFrame and Series
# Here, again, we see that we get NaN if labels are not present in both inputs
df - pd.Series({'a':5, 'b':5, 'e':5, 'f': 5})

In [ ]:
# Normal math operators cannot get an axis argument
# To do this, there are functions for every math operator
df.sub( df.mean(axis=1), axis=0)

# Function Application

In [ ]:
df = pd.DataFrame({'sin': np.arange(0, 5*np.pi, 0.01), 
                   'cos': np.arange(0.5*np.pi, 5.5*np.pi, 0.01)})

# Numpy ufuncs like np.sin operate on every cell
# Here we compute the sin for every cell in the dataframe
df = np.sin(df)

%matplotlib inline
df.plot()

In [ ]:
def iqr(col):
    q1 = col.quantile(.25)
    q3 = col.quantile(.75)
    return q3 - q1 

# df.apply() executes the given function on a whole row or column
df.apply(iqr)

In [ ]:
def somefunc(x):
    return np.abs(x+.25)

# df.applymap() applies the given function for every cell in the DataFrame
df.applymap(somefunc).plot()

## Groups and Aggregations with groupby()

In [ ]:
athletes = pd.read_csv('athletes.csv')
athletes.info()

In [ ]:
# Simply calling groupby returns a GroupBy object 
# This does not calculate anything yet!
g = athletes.groupby('nationality')['weight', 'height']
g

In [ ]:
# Calling an aggregation function on the GroupBy object
# applies the calculation for every group
# and constructs a DataFrame with the results
g.mean()

In [ ]:
# We can select multiple columns to group by
# And we can select a subset of columns to do
g = athletes.groupby(['sport', 'sex'])[['weight', 'height']]

In [ ]:
# Because we selected only 2 columns, this calculation will now be cheaper
g.mean()

# Reshaping Rows and Colums with stack() and unstack()

In [ ]:
m = pd.read_csv('monthly_data.csv')
m

In [ ]:
# Preparation: move the 'YYYY' column into the index
m.set_index('YYYY', inplace=True)
m

In [ ]:
# stack() moves data from rows into a single column
m.stack()

In [ ]:
# stack() also allows quick calculations over all cells
m.stack().sum()

In [ ]:
w = athletes.groupby(['sport', 'sex'])['weight'].mean()
w

In [ ]:
# unstack() takes the inner index level and creates a column for every unique index
# It then moves the data into these columns
w.unstack()

# Reshaping Rows and Colums with pivot()

In [ ]:
p = pd.DataFrame({'id': [823905, 823905,
                         235897, 235897, 235897,
                         983422, 983422],
                  'item': ['prize', 'unit', 
                           'prize', 'unit', 'stock', 
                           'prize', 'stock'],
                  'value': [3.49, 'kg',
                            12.89, 'l', 50,
                            0.49, 4]})
p

In [ ]:
# pivot() moves data from rows into columns
# so that we end up with a wider, shorter DataFrame

# The first argument is the column that will be used for row indices
# The second argument is the column that will be used to create column labels
p.pivot('id', 'item')

In [ ]:
grades = pd.DataFrame([[6, 4, 5], [7, 8, 7], [6, 7, 9], [6, 5, 5], [5, 2, 7]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2', 'test_3'])
grades.reset_index(inplace=True)
grades

In [ ]:
# melt() is the opposite of pivot()
# It moves the data from the rows into a single column
# The column names will show up in a new column called "variable"
grades.melt(id_vars=['index'])

# Combining Datasets

In [ ]:
grades = pd.DataFrame([[6, 4, 5], [7, 8, 7], [6, 7, 9], [6, 5, 5], [5, 2, 7]], 
                       index = ['Mary', 'John', 'Ann', 'Pete', 'Laura'],
                       columns = ['test_1', 'test_2', 'test_3'])
grades

In [ ]:
# Adding a new column -- needs an indexed datastructure (Series)
grades['test_4'] = pd.Series({'John': 5, 'Ann': 8, 'Pete': 9, 'Mary': 7, 'Laura': 10})
grades

In [ ]:
# Adding a row with .loc -- no Series necessary
grades.loc['Bob'] = [2,3,4,5]
grades

In [ ]:
# We can also use append
# But in that case we need a Series with a name (will be used as row index)
new_row = pd.Series({'test_1': 5, 'test_2': 6, 'test_3': 7, 'test_4': 8}, name="Kim")
grades.append(new_row)

In [ ]:
grades['stud_nr'] = [113, 121, 123, 135, 139, 141]
grades = grades[['stud_nr', 'test_1', 'test_2', 'test_3', 'test_4']]
grades

In [ ]:
other = pd.DataFrame([[139, 7, 7],
                       [123, 8, 6],
                       [142, 4, 5],
                       [113, 7, 9],
                       [155, 10, 9],
                       [121, 6, 4]], 
                       columns = ['stud_nr', 'exam1', 'exam2'])
other

In [ ]:
# Merging two DataFrames
# By default this does an inner join on the common column (stud_nr)
grades.merge(other)

In [ ]:
# We can also specify other join types: left, right, outer
grades.merge(other, how='outer')